In [21]:
!pip install -q gradio
!pip install -q torch torchaudio
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q groq
!pip install -q gTTS


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [22]:
import whisper

stt_model = whisper.load_model("base")

def speech_to_text(audio_path):
    result = stt_model.transcribe(audio_path)
    return result["text"]


In [23]:
from groq import Groq
import os

client = Groq(
    api_key="GROQ_API_KEY",
)

def chat_with_groq(text):
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "user", "content": text}
        ],
    )
    return completion.choices[0].message.content

In [24]:
from gtts import gTTS

def text_to_speech(text):
    output_path = "output.mp3"
    tts = gTTS(text=text, lang="en")
    tts.save(output_path)
    return output_path


In [25]:
def voice_ai(audio):
    user_text = speech_to_text(audio)
    ai_text = chat_with_groq(user_text)
    ai_audio = text_to_speech(ai_text)
    return ai_text, ai_audio


In [26]:
import gradio as gr

ui = gr.Interface(
    fn=voice_ai,
    inputs=gr.Audio(type="filepath", label="🎤 Speak"),
    outputs=[
        gr.Textbox(label="🤖 AI Response"),
        gr.Audio(label="🔊 AI Voice")
    ],
    title="Voice-to-Voice AI (100% Free + Open Source)",
    description="Whisper + Groq LLaMA-3 + Google TTS + Gradio"
)

ui.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c4b1bba92bac1e0dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
def speech_to_text(audio_path):
    if audio_path is None:
        return ""

    result = stt_model.transcribe(audio_path)
    text = result.get("text", "").strip()
    return text


In [28]:
from gtts import gTTS

def text_to_speech(text):
    if not text or text.strip() == "":
        text = "Sorry, I could not understand your voice."

    output_path = "output.mp3"
    tts = gTTS(text=text, lang="en")
    tts.save(output_path)
    return output_path


In [29]:
def voice_ai(audio):
    try:
        print("Received audio:", audio)

        user_text = speech_to_text(audio)
        print("User said:", user_text)

        if user_text == "":
            return "I could not hear you properly.", None

        ai_text = chat_with_groq(user_text)
        print("AI response:", ai_text)

        ai_audio = text_to_speech(ai_text)

        return ai_text, ai_audio

    except Exception as e:
        print("ERROR:", str(e))
        return f"Error occurred: {str(e)}", None


In [30]:
inputs=gr.Audio(type="filepath", label="🎤 Speak")
